# Installing required packages
`pyodbc polars import-ipynb`
**REMOVED SERVER NAME AND DB NAME**

# Importing Libraries, Classes from User-defines module

In [1]:
import import_ipynb
from database import SQLServerDB
from data_prepn import PrepareData
import polars as pl
import pandas as pd
import pyarrow

# Checking and creating required DB in SQL Server
databases = ["ecommerce_raw_dataset", "ecommerce_in_dataset"]
obj = SQLServerDB()
obj.connect()
for db in databases:
    sql = f"""IF NOT EXISTS (SELECT * from sys.databases where name = '{db}')
            BEGIN 
                EXEC('CREATE DATABASE {db}');
            END
            """
    obj.execute_queries(sql)
obj.close()
    

importing Jupyter notebook from database.ipynb
importing Jupyter notebook from data_prepn.ipynb
Connected to SQL Server successfully.
Query Executed Successfully:-IF NOT EXISTS (SELECT * from sys.databases where name = 'ecommerce_raw_dataset')
            BEGIN 
                EXEC('CREATE DATABASE ecommerce_raw_dataset');
            END
            
Query Executed Successfully:-IF NOT EXISTS (SELECT * from sys.databases where name = 'ecommerce_in_dataset')
            BEGIN 
                EXEC('CREATE DATABASE ecommerce_in_dataset');
            END
            
Connection closed.


# Read the required CSV files

In [ ]:
# Static File Paths
# file_paths = ['data/data/data/aisles.csv', 'data/data/data/departments.csv', 'data/data/data/order_products.csv', 'data/data/data/orders.csv', 'data/data/data/products.csv']
file_paths = ['data/data/data/aisles.csv', 'data/data/data/departments.csv']
# Reading the required CSV files
data_obj = PrepareData(file_paths)
dataframes = data_obj.read_file()
# print(dataframes)

# Perform Transformation for RAW TABLES
**All cols must be of type STRING**

In [ ]:
# using Class method to change col type from int to str
raw_dfs = data_obj.int_to_str(dataframes)
# print(raw_dfs['aisles'])

# Creating tables in RAW Database

In [ ]:
# Creating the SQL queries to create the table in RAW DATASET
sql_queries = []
for key, df in raw_dfs.items():
    col_name = data_obj.get_col_name(df)
    values = ','.join([f"{str} varchar(1000)" for str in col_name])
    sql = f"""
            IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{key}')
            BEGIN
                CREATE TABLE {key} ({values});
            END;
        """
    sql_queries.append(sql)

# print(sql_queries)

# Running the queries in Already made DB
obj_raw_table = SQLServerDB()
obj_raw_table.connect()

try:
    for query in sql_queries:
        obj_raw_table.execute_queries(query)
except Exception as e:
    print(f"Exception occured:{e}")
finally:
    print("All raw tables created")

# Close the connection
obj_raw_table.close()



# Insert data into RAW Tables

In [ ]:
for key, df in raw_dfs.items():
    # Convert Polars DataFrame to Pandas DataFrame
    pandas_df = df.to_pandas()
    
    # Get column names dynamically
    columns = pandas_df.columns
    column_names = ', '.join(columns)
    placeholders = ', '.join(['?'] * len(columns))
    insert_query = f"INSERT INTO {key} ({column_names}) VALUES ({placeholders})"
    
    # Convert DataFrame to list of tuples
    data = pandas_df.values.tolist()

    insert_data_into_raw = SQLServerDB()
    insert_data_into_raw.connect()
    insert_data_into_raw.insert_data(insert_query, data)    
    insert_data_into_raw.close()
    
    

# Create the Insight tables and do the analysis on following questions:-
## Steps:-
**Create table > Perform transformation > Check for Null Values(only for INT cols) > Fill Nulls(default 0) > Insert the data into the respective tables > perform below analysis**

1. Create a temporary table that joins the orders, order_products, and products tables to get information about each order, including the products that were purchased and their department and aisle information.
2. Create a temporary table that groups the orders by product and finds the total number of times each product was purchased, the total number of times each product was reordered, and the average number of times each product was added to a cart.
3. Create a temporary table that groups the orders by department and finds the total number of products purchased, the total number of unique products purchased, the total number of products purchased on weekdays vs weekends, and the average time of day that products in each department are ordered.
4. Create a temporary table that groups the orders by aisle and finds the top 10 most popular aisles, including the total number of products purchased and the total number of unique products purchased from each aisle.
5. Combine the information from the previous temporary tables into a final table that shows the product ID, product name, department ID, department name, aisle ID, aisle name, total number of times purchased, total number of times reordered, average number of times added to cart, total number of products purchased, total number of unique products purchased, total number of products purchased on weekdays, total number of products purchased on weekends, and average time of day products are ordered in each department.

In [3]:
import os
# Creating the Insight table in Insight dataset
ddl_sql_path = 'insight_tbl_ddl_queries'
insight_obj = SQLServerDB()
insight_obj.connect()

for file_name in os.listdir(ddl_sql_path):
    com_file_path = f"{ddl_sql_path}/{file_name}"
    
    if os.path.isfile(com_file_path):
        with open(com_file_path, 'r') as f:
            sql = f.read()
            insight_obj.execute_queries(sql)
            
insight_obj.close()

Connected to SQL Server successfully.
Error while executing the query:- ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'aisles' in the database. (2714) (SQLExecDirectW)")
Error while executing the query:- ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'departments' in the database. (2714) (SQLExecDirectW)")
Error while executing the query:- ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'orders' in the database. (2714) (SQLExecDirectW)")
Query Executed Successfully:-CREATE TABLE order_products (
        order_id INTEGER,
        product_id INTEGER,
        add_to_cart_order INTEGER,
        reordered INTEGER,
        PRIMARY KEY (order_id, product_id),
        FOREIGN KEY (order_id) REFERENCES orders (order_id),
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
Error while executing the que

# Transforming the data as per the schema

In [5]:
raw_obj = SQLServerDB()
insight = PrepareData(None)
raw_obj.connect()

str_to_int_dict = {
    'aisles': ['aisle_id'],
    'departments':['department_id'],
    'order_products':['order_id', 'product_id', 'add_to_cart_order', 'reordered'],
    'orders':['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day'],
    'products':['product_id', 'aisle_id', 'department_id']
}

sql = "SELECT * FROM INFORMATION_SCHEMA.TABLES"
df = raw_obj.read_the_table(sql)
table_names = df.select(pl.col("TABLE_NAME"))
tbl_name_list = table_names["TABLE_NAME"].to_list()

for tbl_name in tbl_name_list:
    sql = f"SELECT * FROM {tbl_name}"
    get_data = raw_obj.read_the_table(sql)
    if tbl_name == 'orders':
        get_data = insight.fill_nulls(get_data,'days_since_prior_order').select("*").with_columns(pl.col('days_since_prior_order').cast(pl.Float64).cast(pl.Int64))
        get_data = insight.str_to_int(get_data, str_to_int_dict.get(tbl_name))
        final_in_df = get_data.drop(pl.col("eval_set"))
    else:
        final_in_df =  insight.str_to_int(get_data, str_to_int_dict.get(tbl_name))
    print(final_in_df)
    # Convert Polars DataFrame to Pandas DataFrame
    pandas_df = final_in_df.to_pandas()
    
    # Get column names dynamically
    columns = pandas_df.columns
    column_names = ', '.join(columns)
    placeholders = ', '.join(['?'] * len(columns))
    insert_query = f"INSERT INTO {tbl_name} ({column_names}) VALUES ({placeholders})"
    
    # Convert DataFrame to list of tuples
    data = pandas_df.values.tolist()
    
    insert_data_into_raw = SQLServerDB()
    insert_data_into_raw.connect()
    insert_data_into_raw.insert_data(insert_query, data)    
    insert_data_into_raw.close()
    # print(get_data)
raw_obj.close()

Connected to SQL Server successfully.
shape: (32_434_489, 4)
┌──────────┬────────────┬───────────────────┬───────────┐
│ order_id ┆ product_id ┆ add_to_cart_order ┆ reordered │
│ ---      ┆ ---        ┆ ---               ┆ ---       │
│ i64      ┆ i64        ┆ i64               ┆ i64       │
╞══════════╪════════════╪═══════════════════╪═══════════╡
│ 28       ┆ 27548      ┆ 9                 ┆ 1         │
│ 93       ┆ 39408      ┆ 11                ┆ 0         │
│ 146      ┆ 26348      ┆ 6                 ┆ 1         │
│ 213      ┆ 26165      ┆ 11                ┆ 1         │
│ 265      ┆ 41658      ┆ 4                 ┆ 1         │
│ …        ┆ …          ┆ …                 ┆ …         │
│ 3421083  ┆ 39678      ┆ 6                 ┆ 1         │
│ 3421083  ┆ 11352      ┆ 7                 ┆ 0         │
│ 3421083  ┆ 4600       ┆ 8                 ┆ 0         │
│ 3421083  ┆ 24852      ┆ 9                 ┆ 1         │
│ 3421083  ┆ 5020       ┆ 10                ┆ 1         │
└──────────